In [1]:
# import pandas and numpy
import pandas as pd
import numpy as np

       ################# 
         #Import necessary package for Scrapping web pag
       ###################### 

#After installing beautifulsoup wip 
#pip install beautifulsoup4
from bs4 import BeautifulSoup
# Install lxml with:  pip install lxml qfter installinng import the package
from IPython.display import HTML, display
import requests

        #######################################
        #Import the necessary dependancy for the geolocalise
        #######################################

#install geopy with :pip3 install geopy   
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#install geopy with :pip3 install folium
import folium # map rendering library
#install json_normalize
from pandas.io.json import json_normalize

        ################# 
         #Scrapping web page
        #####################

from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

        ###############################################################
        ###### Import necessary  library to make instogramme
        #############################################################

import matplotlib.pyplot as plt
%matplotlib inline 
#print('Matplotlib version'+plt.__version__ )

In [2]:
 # Code for import the data frame and the table contant in the page
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df.columns = ['Postcode', 'Borough', 'Neighbourhood'] # Give colums Na
#df[0:3]

In [3]:
#Drop row with not assigmed
df = df[df.Borough != 'Not assigned'] 
df=df.drop(df.index[0]) # Drop first row
#df[0:3]

In [12]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].agg(','.join).reset_index() # marge only the Neighbourhod coll in dependancy of Postcode ans Borough
df[0:3]

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"


In [13]:
# Select the row where the in Neighbourhood there is the value Not assigned and declare a new dataframe to show how the change is done 
#df= df.loc[df['Neighbourhood'] == 'Not assigned']
#df[0:86]
df.loc[df.Neighbourhood =='Not assigned', 'Neighbourhood']=df.Borough# replacement of pandas cell values with cell values of another row and print the result
#df[0:86]

In [14]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data') #Import Data necessary for the geolocatisation
df_geo.columns = ['Postcode', 'Latitude', 'Longitude'] #Give a name to the all the coll present in the dataframe df_ge0
df_geo[0:3]

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


In [15]:
# make a fusion from the basic doccumentation and the latitude, longitude, Postcode, basis on Postcode
df_pos = pd.merge(df, df_geo, on=['Postcode'], how='inner')
#df_pos[0:3]

In [37]:
#Make a reorganisation from all the colls present in the dataframe
df_reorganisation = df_pos[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()
tnew=df_reorganisation[0:3]
tnew
#df_reorganisation['Latitude']

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711


### Print coordonate from Toronto Canada

In [25]:
address = 'Toronto, Canada' #Make a declaration from the adress Toronto Canada
# Code to find the coordinator from toronto and canada
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The l coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The l coordinate of the City of Toronto are 43.653963, -79.387207.


/home/loth/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(df_reorganisation['Latitude'], df_reorganisation['Longitude'], df_reorganisation['Borough'], df_reorganisation['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)    
#map_toronto

In [28]:
df_t1 = df_reorganisation[df_reorganisation['Borough'].str.contains('Toronto')] # Define a new dataframe with only Toronto as Borough
to_data = df_t1.reset_index(drop=True) # Help to put the index declaration in the right order 0,1,2,3,4
to_data [0:5]
#df_t4

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,East Toronto,"The Danforth West,Riverdale",M4K,43.679557,-79.352188
2,East Toronto,"The Beaches West,India Bazaar",M4L,43.668999,-79.315572
3,East Toronto,Studio District,M4M,43.659526,-79.340923
4,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790


### Explore the first neighbourhood

In [29]:
to_data.loc[10, 'Neighbourhood']

'Rosedale'

In [30]:
neighbourhood_latitude = to_data.loc[10, 'Latitude']   # value of the neighbourhood latitude 
neighbourhood_longitude = to_data.loc[10, 'Longitude'] # value of the neighbourhood longitude 

neighbourhood_name = to_data.loc[10, 'Neighbourhood'] # giver a name to the neighbourhood 

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


# Scraping the different Foursquare parameters

In [41]:
# https://stackoverflow.com/questions/57381511/extracting-specific-venue-categories-from-foursquare-api-using-python-code
name
LIMIT = 10
radius = 500
CLIENT_ID = '3WGIGW2JP1DAQABSATJENRP25A3THDBEC303QEHM0KPKGKMT'     # your Foursquare ID
CLIENT_SECRET = 'RCOBC0XDX30OHNM402UL5AXMRGHCXZML042LJ2EEERQSRIFW' # your Foursquare Secret
VERSION = '20180605' # Version declaration 
latitudes =  tnew['Latitude']
longitudes = tnew['Longitude']
nearby_venues=[]
venues_list = []
def getNearbyVenues(names, latitudes, longitudes, radius=2000, LIMIT=50, categoryId='52f2ab2ebcbc57f1066b8b46'):

# Make a request from Foursquare to get the
for lat, lng in zip(latitudes,longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
    url
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for v in results:
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'])]) 

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood',
                                 'Neighborhood Latitude',
                                 'Neighborhood Longitude',
                                 'Venue', 
                                 'Venue Latitude',
                                 'Venue Longitude', 
                                 'Venue Category']

In [42]:
#nearby_venues for 
nearby_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,[],43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,[],43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,[],43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,[],43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,[],43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,[],43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,[],43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,[],43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,[],43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,[],43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


# Necessary for testing the different parameters

In [32]:
LIMIT = 10
radius = 500
CLIENT_ID = '3WGIGW2JP1DAQABSATJENRP25A3THDBEC303QEHM0KPKGKMT' # your Foursquare ID
CLIENT_SECRET = 'RCOBC0XDX30OHNM402UL5AXMRGHCXZML042LJ2EEERQSRIFW' # your Foursquare Secret
VERSION = '20180605' # Version declaration 
neighbourhood_latitude = 43.806686
neighbourhood_longitude = -79.194353

# Make a request from Foursquare to get the
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url3
results4 = requests.get(url3).json()
results4
# assign relevant part of JSON to venues
venues4 = results4['response']['groups'][0]['items']
# tranform venues into a dataframe
dataframe3 = json_normalize(venues4)
dataframe3

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4bb6b9446edc76b0d771311c-0,"[{'primary': True, 'name': 'Fast Food Restaura...",4bb6b9446edc76b0d771311c,CA,Toronto,Canada,Morningside & Sheppard,387,"[Toronto ON, Canada]","[{'lng': -79.19905558052072, 'label': 'display...",43.807448,-79.199056,ON,Wendy's,0,[]
